<a href="https://colab.research.google.com/github/emnordi/ce888/blob/master/Project2/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import os
import io
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

os.environ['KAGGLE_USERNAME'] = "emilnordin" # username from the json file

os.environ['KAGGLE_KEY'] = "3f9baf0776ca705fb3dbe29ef5e7caab" # key from the json file

DOWNLOADING AND IMPORTING DATASETS

In [2]:
!kaggle datasets download -d uciml/mushroom-classification

  0% 0.00/34.2k [00:00<?, ?B/s]
100% 34.2k/34.2k [00:00<00:00, 27.0MB/s]


In [3]:
!kaggle datasets download -d emoneyanalytics/unbalancedrisk

  0% 0.00/115k [00:00<?, ?B/s]
100% 115k/115k [00:00<00:00, 52.0MB/s]


In [4]:
!kaggle datasets download -d subediaarjun/unbalanced-fraud-detection-dataset

  0% 0.00/2.57M [00:00<?, ?B/s]
100% 2.57M/2.57M [00:00<00:00, 83.5MB/s]


In [5]:
#This dataset happened to be in excel format requiring some extra steps
!unzip unbalanced-fraud-detection-dataset.zip

Archive:  unbalanced-fraud-detection-dataset.zip
  inflating: Fraud Detection Sample data.xlsx  


In [0]:
data_mush = pd.read_csv('mushroom-classification.zip', compression='zip', header=0, sep=',', quotechar='"', encoding = "utf-8")

In [0]:
#Downsample to get 65% imbalance
mush_p_ind = data_mush[data_mush['class'] == 'p'].index
to_del = mush_p_ind[0:int(len(mush_p_ind)*0.43)]
data_mush_imbalanced = data_mush.drop(index=to_del)

#Convert string data to ints
data_mush_imbalanced_ = pd.get_dummies(data_mush_imbalanced)
data_mush_imbalanced_ = data_mush_imbalanced_.drop('class_e', axis=1)

In [0]:
data_risk = pd.read_csv('unbalancedrisk.zip', compression='zip', header=0, sep=',', quotechar='"', encoding = "utf-8")

In [0]:
data_fraud = pd.read_excel("Fraud Detection Sample data.xlsx", sheet_name = 'credit_card_fraud_sample', header = 0)

PRINTING DATASETS

In [10]:
print(data_risk)

        id  f1        f2  f3   f4  ...   f31         f32    f33      f34  obj1
0        1   1   17.2131   1  1.0  ...  13.0   32.766370    NaN      NaN     0
1        2  35    0.1747   1  1.0  ...  23.0  210.567669    NaN      NaN     0
2        3   1   35.3943   1  NaN  ...  16.0    0.000111    NaN      NaN     1
3        4   1    4.4873   1  1.0  ...   7.0    9.380889    NaN      NaN     0
4        5   1       NaN   1  NaN  ...  20.0   28.089135    NaN      NaN     0
...    ...  ..       ...  ..  ...  ...   ...         ...    ...      ...   ...
4523  4524  29    5.7576   1  1.0  ...  32.0  236.170700    NaN  63.0541     0
4524  4525   2  107.7430   1  1.0  ...  27.0    2.799800    NaN      NaN     1
4525  4526   1    1.9364   1  1.0  ...   NaN  205.806400    NaN  15.9357     1
4526  4527   1    7.6795   1  3.0  ...  10.0    1.658700  533.0      NaN     0
4527  4528   1   80.0871   1  1.0  ...   NaN    2.980200    NaN      NaN     1

[4528 rows x 36 columns]


In [11]:
print(data_fraud)

          ID        V1        V2        V3  ...       V27       V28  Amount  Class
0     221547 -1.191668  0.428409  1.640028  ... -0.319964 -0.017444   27.44      0
1     184524  1.966614 -0.450087 -1.228586  ... -0.039155 -0.071314   35.95      0
2      91201  1.528452 -1.296191 -0.890677  ...  0.042997  0.025853   28.40      0
3      26115 -0.774614  1.100916  0.679080  ...  0.282940  0.123856   43.20      0
4     201292 -1.075860  1.361160  1.496972  ...  0.951130  0.158996    0.83      1
...      ...       ...       ...       ...  ...       ...       ...     ...    ...
7295   96789 -3.593476  0.781442 -1.822448  ... -1.478767  0.722673  101.50      1
7296   18809  0.923764  0.344048 -2.880004  ...  0.489035 -0.049729   30.30      1
7297  197586 -0.361428  1.133472 -2.971360  ... -0.001250 -0.182751  480.72      1
7298  184379 -1.141559  1.927650 -3.905356  ... -0.146102  0.076211   25.00      1
7299  145800 -0.419820 -1.155978 -2.092516  ... -0.021363  0.129096  451.27      1

[73

In [12]:
print(data_mush_imbalanced_)

      class_p  cap-shape_b  cap-shape_c  ...  habitat_p  habitat_u  habitat_w
1           0            0            0  ...          0          0          0
2           0            1            0  ...          0          0          0
4           0            0            0  ...          0          0          0
5           0            0            0  ...          0          0          0
6           0            1            0  ...          0          0          0
...       ...          ...          ...  ...        ...        ...        ...
8119        0            0            0  ...          0          0          0
8120        0            0            0  ...          0          0          0
8121        0            0            0  ...          0          0          0
8122        1            0            0  ...          0          0          0
8123        0            0            0  ...          0          0          0

[6441 rows x 116 columns]


DISPLAY AMOUNT OF ROWS EACH CLASS HAS, AND IMBALANCE OF DATASETS

In [13]:
print(data_mush_imbalanced_['class_p'].value_counts())
print("Imbalance:", data_mush_imbalanced_['class_p'].value_counts()[0]/(data_mush_imbalanced_['class_p'].value_counts()[1]+(data_mush_imbalanced_['class_p'].value_counts()[0])))

0    4208
1    2233
Name: class_p, dtype: int64
Imbalance: 0.6533147026859183


In [14]:
print(data_risk['obj1'].value_counts())
print("Imbalance:", data_risk['obj1'].value_counts()[0]/(data_risk['obj1'].value_counts()[1]+(data_risk['obj1'].value_counts()[0])))

0    3694
1     834
Name: obj1, dtype: int64
Imbalance: 0.8158127208480566


In [15]:
print(data_fraud['Class'].value_counts())
print("Imbalance:", data_fraud['Class'].value_counts()[0]/(data_fraud['Class'].value_counts()[1]+(data_fraud['Class'].value_counts()[0])))

0    6972
1     328
Name: Class, dtype: int64
Imbalance: 0.955068493150685


BASE RESULTS

In [0]:
tree_clf = tree.DecisionTreeClassifier(random_state=0)
rf_clf = RandomForestClassifier(max_depth=2, random_state=0)

In [17]:
#Mushroom dataset base scores DT
feat_mush = data_mush_imbalanced_.drop('class_p', axis=1)
lab_mush = data_mush_imbalanced_['class_p']

print("Accuracy:", cross_val_score(tree_clf, feat_mush, lab_mush, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(tree_clf, feat_mush, lab_mush, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(tree_clf, feat_mush, lab_mush, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(tree_clf, feat_mush, lab_mush, cv=10, scoring='f1').mean())

Accuracy: 0.9996899224806202
Precision: 1.0
Recall: 0.9991071428571429
F1-Score: 0.9995515695067265


In [24]:
#Mushroom dataset base scores RF
print("Accuracy:", cross_val_score(rf_clf, feat_mush, lab_mush, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(rf_clf, feat_mush, lab_mush, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(rf_clf, feat_mush, lab_mush, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(rf_clf, feat_mush, lab_mush, cv=10, scoring='f1').mean())

Accuracy: 0.9394662718474649
Precision: 0.9823529411764707
Recall: 0.8455557335041641
F1-Score: 0.8938599999924101


In [19]:
#Risk dataset base scores DT
feat_risk = data_risk.drop('obj1', axis=1)
for col in feat_risk.columns:
  if feat_risk[col].isnull().any():
    feat_risk[col] = feat_risk[col].fillna(feat_risk[col].median())

lab_risk = data_risk['obj1']

print("Accuracy:", cross_val_score(tree_clf, feat_risk, lab_risk, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(tree_clf, feat_risk, lab_risk, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(tree_clf, feat_risk, lab_risk, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(tree_clf, feat_risk, lab_risk, cv=10, scoring='f1').mean())


Accuracy: 0.5392003164742425
Precision: 0.16305450371332567
Recall: 0.32172977624784854
F1-Score: 0.1982397011891827


In [28]:
#Risk dataset base scores RF
print("Accuracy:", cross_val_score(rf_clf, feat_risk, lab_risk, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(rf_clf, feat_risk, lab_risk, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(rf_clf, feat_risk, lab_risk, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(rf_clf, feat_risk, lab_risk, cv=10, scoring='f1').mean())

Accuracy: 0.8087333216120651


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined a

Precision: 0.013636363636363636
Recall: 0.007228915662650602
F1-Score: 0.009448818897637793


In [20]:
#Fraud dataset base scores DT
feat_fraud = data_fraud.drop('Class', axis=1)
lab_fraud = data_fraud['Class']

print("Accuracy:", cross_val_score(tree_clf, feat_fraud, lab_fraud, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(tree_clf, feat_fraud, lab_fraud, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(tree_clf, feat_fraud, lab_fraud, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(tree_clf, feat_fraud, lab_fraud, cv=10, scoring='f1').mean())

Accuracy: 0.9784931506849315
Precision: 0.7837433451305875
Recall: 0.7607007575757576
F1-Score: 0.7430508668262178


In [29]:
#Fraud dataset base scores RD
print("Accuracy:", cross_val_score(rf_clf, feat_fraud, lab_fraud, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(rf_clf, feat_fraud, lab_fraud, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(rf_clf, feat_fraud, lab_fraud, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(rf_clf, feat_fraud, lab_fraud, cv=10, scoring='f1').mean())

Accuracy: 0.9864383561643836
Precision: 0.9960000000000001
Recall: 0.6997159090909092
F1-Score: 0.7979241651083359


DOWNSAMPLING THE DATASETS AND RETRIEVING NEW PERFORMANCE RESULTS

In [40]:
#Downsample MUSHROOM DATASET to get 50%
mush_p_ind_ = data_mush_imbalanced_[data_mush_imbalanced_['class_p'] == 0].index
to_del = mush_p_ind_[0:int(len(mush_p_ind_)*0.46)]
data_mush_imbalanced_test = data_mush_imbalanced_.drop(index=to_del)

print(data_mush_imbalanced_test['class_p'].value_counts())
print("Imbalance:", data_mush_imbalanced_test['class_p'].value_counts()[0]/(data_mush_imbalanced_test['class_p'].value_counts()[1]+(data_mush_imbalanced_test['class_p'].value_counts()[0])))

0    2273
1    2233
Name: class_p, dtype: int64
Imbalance: 0.5044385264092321


In [41]:
#MUSHROOM DATASET downsampling scores DT
feat_mush_test = data_mush_imbalanced_test.drop('class_p', axis=1)
lab_mush_test = data_mush_imbalanced_test['class_p']

print("Accuracy:", cross_val_score(tree_clf, feat_mush_test, lab_mush_test, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(tree_clf, feat_mush_test, lab_mush_test, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(tree_clf, feat_mush_test, lab_mush_test, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(tree_clf, feat_mush_test, lab_mush_test, cv=10, scoring='f1').mean())

Accuracy: 0.9995565410199557
Precision: 1.0
Recall: 0.9991031390134528
F1-Score: 0.9995495495495496


In [42]:
#MUSHROOM DATASET downsampling scores RF
print("Accuracy:", cross_val_score(rf_clf, feat_mush_test, lab_mush_test, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(rf_clf, feat_mush_test, lab_mush_test, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(rf_clf, feat_mush_test, lab_mush_test, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(rf_clf, feat_mush_test, lab_mush_test, cv=10, scoring='f1').mean())

Accuracy: 0.9640517368810052
Precision: 0.9780786764002677
Recall: 0.952017937219731
F1-Score: 0.9623570737843696


In [54]:
#Downsample RISK DATASET to get 50%
data_risk_ind_ = data_risk[data_risk['obj1'] == 0].index
r_to_del = data_risk_ind_[0:int(len(data_risk_ind_)*0.774)]
data_risk_test = data_risk.drop(index=r_to_del)

print(data_risk_test['obj1'].value_counts())
print("Imbalance:", data_risk_test['obj1'].value_counts()[0]/(data_risk_test['obj1'].value_counts()[1]+(data_risk_test['obj1'].value_counts()[0])))

0    835
1    834
Name: obj1, dtype: int64
Imbalance: 0.5002995805871779


In [55]:
#Risk dataset downsampled scores DT
feat_risk_test = data_risk_test.drop('obj1', axis=1)
for col in feat_risk_test.columns:
  if feat_risk_test[col].isnull().any():
    feat_risk_test[col] = feat_risk_test[col].fillna(feat_risk_test[col].median())

lab_risk_test = data_risk_test['obj1']

print("Accuracy:", cross_val_score(tree_clf, feat_risk_test, lab_risk_test, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(tree_clf, feat_risk_test, lab_risk_test, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(tree_clf, feat_risk_test, lab_risk_test, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(tree_clf, feat_risk_test, lab_risk_test, cv=10, scoring='f1').mean())

Accuracy: 0.6110778443113772
Precision: 0.4910334294672273


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recall: 0.7060240963855422
F1-Score: 0.5732171901186317


In [56]:
#Risk dataset downsample scores RF
print("Accuracy:", cross_val_score(rf_clf, feat_risk_test, lab_risk_test, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(rf_clf, feat_risk_test, lab_risk_test, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(rf_clf, feat_risk_test, lab_risk_test, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(rf_clf, feat_risk_test, lab_risk_test, cv=10, scoring='f1').mean())

Accuracy: 0.7697713007719501
Precision: 0.8020669323477445
Recall: 0.7228915662650602
F1-Score: 0.6687959031965699


In [0]:
#Downsample FRAUD DATASET to get 50%
data_fraud_ind_ = data_fraud[data_fraud['Class'] == 0].index
f_to_del = data_fraud_ind_[0:int(len(data_fraud_ind_)*0.952)]
data_fraud_test = data_fraud.drop(index=f_to_del)

print(data_fraud_test['Class'].value_counts())
print("Imbalance:", data_fraud_test['Class'].value_counts()[0]/(data_fraud_test['Class'].value_counts()[1]+(data_fraud_test['Class'].value_counts()[0])))

In [62]:
#Fraud dataset base scores DT
feat_fraud_test = data_fraud_test.drop('Class', axis=1)
lab_fraud_test = data_fraud_test['Class']

print("Accuracy:", cross_val_score(tree_clf, feat_fraud_test, lab_fraud_test, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(tree_clf, feat_fraud_test, lab_fraud_test, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(tree_clf, feat_fraud_test, lab_fraud_test, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(tree_clf, feat_fraud_test, lab_fraud_test, cv=10, scoring='f1').mean())

Accuracy: 0.8630936227951154
Precision: 0.8746967474541003
Recall: 0.8481060606060605
F1-Score: 0.8452735455304451


In [63]:
#Fraud dataset base scores RD
print("Accuracy:", cross_val_score(rf_clf, feat_fraud_test, lab_fraud_test, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(rf_clf, feat_fraud_test, lab_fraud_test, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(rf_clf, feat_fraud_test, lab_fraud_test, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(rf_clf, feat_fraud_test, lab_fraud_test, cv=10, scoring='f1').mean())

Accuracy: 0.8948213478064224
Precision: 0.990073145245559
Recall: 0.7964962121212121
F1-Score: 0.859661349183995


OVERSAMPLING AND TEST RESULTS

In [70]:
#OVERSAMPLING MUSHROOM DATASET
max_ = data_mush_imbalanced_['class_p'].value_counts().max()

temp = [data_mush_imbalanced_]
for class_index, group in data_mush_imbalanced_.groupby('class_p'):
    temp.append(group.sample(max_-len(group), replace=True))
data_mush_over = pd.concat(temp)

print(data_mush_over['class_p'].value_counts())
print("Imbalance:", data_mush_over['class_p'].value_counts()[0]/(data_mush_over['class_p'].value_counts()[1]+(data_mush_over['class_p'].value_counts()[0])))

1    4208
0    4208
Name: class_p, dtype: int64
Imbalance: 0.5


In [73]:
#MUSHROOM DATASET oversampling scores DT
feat_mush_over = data_mush_over.drop('class_p', axis=1)
lab_mush_over = data_mush_over['class_p']

print("Accuracy:", cross_val_score(tree_clf, feat_mush_over, lab_mush_over, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(tree_clf, feat_mush_over, lab_mush_over, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(tree_clf, feat_mush_over, lab_mush_over, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(tree_clf, feat_mush_over, lab_mush_over, cv=10, scoring='f1').mean())

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


In [74]:
#MUSHROOM DATASET downsampling scores RF
print("Accuracy:", cross_val_score(rf_clf, feat_mush_over, lab_mush_over, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(rf_clf, feat_mush_over, lab_mush_over, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(rf_clf, feat_mush_over, lab_mush_over, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(rf_clf, feat_mush_over, lab_mush_over, cv=10, scoring='f1').mean())

Accuracy: 0.9396327186558249
Precision: 0.9804210526315789
Recall: 0.9013827621309808
F1-Score: 0.9358573084322905


In [71]:
#OVERSAMPLING RISK DATASET
max_ = data_risk['obj1'].value_counts().max()

temp = [data_risk]
for class_index, group in data_risk.groupby('obj1'):
    temp.append(group.sample(max_-len(group), replace=True))
data_risk_over = pd.concat(temp)

print(data_risk_over['obj1'].value_counts())
print("Imbalance:", data_risk_over['obj1'].value_counts()[0]/(data_risk_over['obj1'].value_counts()[1]+(data_risk_over['obj1'].value_counts()[0])))

1    3694
0    3694
Name: obj1, dtype: int64
Imbalance: 0.5


In [75]:
#Risk dataset oversampled scores DT
feat_risk_over = data_risk_over.drop('obj1', axis=1)
for col in feat_risk_over.columns:
  if feat_risk_over[col].isnull().any():
    feat_risk_over[col] = feat_risk_over[col].fillna(feat_risk_over[col].median())

lab_risk_over = data_risk_over['obj1']

print("Accuracy:", cross_val_score(tree_clf, feat_risk_over, lab_risk_over, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(tree_clf, feat_risk_over, lab_risk_over, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(tree_clf, feat_risk_over, lab_risk_over, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(tree_clf, feat_risk_over, lab_risk_over, cv=10, scoring='f1').mean())

Accuracy: 0.5692835480452233
Precision: 0.5419794462150709
Recall: 0.9937669376693767
F1-Score: 0.6999812426287944


In [76]:
#RISK DATASET oversampling scores RF
print("Accuracy:", cross_val_score(rf_clf, feat_risk_over, lab_risk_over, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(rf_clf, feat_risk_over, lab_risk_over, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(rf_clf, feat_risk_over, lab_risk_over, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(rf_clf, feat_risk_over, lab_risk_over, cv=10, scoring='f1').mean())

Accuracy: 0.40643603932656375
Precision: 0.43695322804157116
Recall: 0.5684911741009302
F1-Score: 0.4917442664733766


In [72]:
#OVERSAMPLING FRAUD DATASET
max_ = data_fraud['Class'].value_counts().max()

temp = [data_fraud]
for class_index, group in data_fraud.groupby('Class'):
    temp.append(group.sample(max_-len(group), replace=True))
data_fraud_over = pd.concat(temp)

print(data_fraud_over['Class'].value_counts())
print("Imbalance:", data_fraud_over['Class'].value_counts()[0]/(data_fraud_over['Class'].value_counts()[1]+(data_fraud_over['Class'].value_counts()[0])))

1    6972
0    6972
Name: Class, dtype: int64
Imbalance: 0.5


In [77]:
#Fraud dataset oversampling scores DT
feat_fraud_over = data_fraud_over.drop('Class', axis=1)
lab_fraud_over = data_fraud_over['Class']

print("Accuracy:", cross_val_score(tree_clf, feat_fraud_over, lab_fraud_over, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(tree_clf, feat_fraud_over, lab_fraud_over, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(tree_clf, feat_fraud_over, lab_fraud_over, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(tree_clf, feat_fraud_over, lab_fraud_over, cv=10, scoring='f1').mean())

Accuracy: 0.9944060823909947
Precision: 0.9889488840732987
Recall: 1.0
F1-Score: 0.9944405666464208


In [78]:
#Fraud dataset oversampling scores RF
print("Accuracy:", cross_val_score(rf_clf, feat_fraud_over, lab_fraud_over, cv=10, scoring='accuracy').mean())
print("Precision:", cross_val_score(rf_clf, feat_fraud_over, lab_fraud_over, cv=10, scoring='precision').mean())
print("Recall:", cross_val_score(rf_clf, feat_fraud_over, lab_fraud_over, cv=10, scoring='recall').mean())
print("F1-Score:", cross_val_score(rf_clf, feat_fraud_over, lab_fraud_over, cv=10, scoring='f1').mean())

Accuracy: 0.8916370723479531
Precision: 0.9913714215462047
Recall: 0.7901643556297354
F1-Score: 0.8793354174840052


CREATE 10 FOLDS OF EACH CLASS

In [30]:
#MUSHROOM DATASET
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(feat_mush, lab_mush)
print(skf)

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)


In [0]:
for x_ind, y_ind in skf.split(feat_mush, lab_mush):
  #print(feat_risk.loc[x_ind])
  #print(lab_risk.loc[y_ind])
  pass

In [134]:
#RISK DATASET
skf1 = StratifiedKFold(n_splits=10)
skf1.get_n_splits(feat_risk, lab_risk)
print(skf1)

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)


In [0]:
for x_ind, y_ind in skf1.split(feat_risk, lab_risk):
  #print(feat_risk.loc[x_ind])
  #print(lab_risk.loc[y_ind])
  pass

In [136]:
#FRAUD DATASET
skf2 = StratifiedKFold(n_splits=10)
skf2.get_n_splits(feat_fraud, lab_fraud)
print(skf2)

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)


In [0]:
for x_ind, y_ind in skf2.split(feat_fraud, lab_fraud):
  #print(feat_fraud.loc[x_ind])
  #print(lab_fraud.loc[y_ind])
  pass